In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

from ppg_project.training_pipeline import XGBoostTrainingPipeline
from ppg_project.dtypes import XGBoostTrainingPipelineConfig

In [ ]:

config = XGBoostTrainingPipelineConfig(
    data_path="../data",
    validation_strategy="kfold",
    num_folds=5,
    features_list = ("features","peak","spectrogram",))
pipe = XGBoostTrainingPipeline(config)

In [ ]:
hyper_params = {
    "max_depth": [3,4,5,6],
    'min_child_weight': [3,6,9],
    'subsample': [1.0],
    'colsample_bytree': [0.2,0.4,0.5,0.6],
}
results_confg, best_config = pipe.hyperparameter_tuning(hyper_params)


In [ ]:
results_confg.drop(columns=["feature_importances","fold"], inplace=True)
results_confg.groupby("params").mean().sort_values("r2", ascending=False)

In [ ]:
pipe = XGBoostTrainingPipeline(best_config)
results = pipe.run()

In [ ]:
results[["rmse","rmsle","mae","r2"]].aggregate([np.mean, np.std])

In [ ]:
import matplotlib.pyplot as plt 

df_feat = pd.json_normalize(results.feature_importances)
list_col_features = ["spectrogram_mean","spectrogram_std"]
for feature in list_col_features:
    list_col = [col for col in df_feat.columns if feature in col]
    print(feature,list_col)
    if list_col:
        df_feat[f"mean_{feature}"] = df_feat[list_col].mean(axis=1)
        df_feat.drop(columns=list_col, inplace=True)

mean_values = df_feat.mean()
std_values = df_feat.std()

# Plot bar plot
fig, ax = plt.subplots()

# Bar plot for mean values
mean_values.plot(kind='barh', yerr=std_values, ax=ax, capsize=4, color='lightblue', edgecolor='black')

# Set plot labels and title
ax.set_ylabel('Mean Value')
ax.set_title('Feature importance with Mean and Std')

# Show the plot
plt.show()